# Updating Data

It would be impractical for data to be placed in a table and then never be amended again or not to have the ability to edit what we are storing.  In this scenario our only option would be to drop the table and re-create a new one with correct data each time, which would be less than performant and unacceptable to almost every business.

Therefore SQL provides the UPDATE keyword in order to allow us to edit our data.  This does not act on whole rows but simply amends the item or items of data we request.  As such, it is an in-place update.  It does not do a Delete and Insert for the row of data as a whole.

It is also worth mentioning that SQL Server (as with all database systems using the SQL language) work on the principle of Sets rather than Rows.  When coding for UPDATE statements we should think about updating whole sets of data at a time where possible rather than row by row.  We will show why this is the case with a demo later on.

Another critical note that needs to be emphasised at the very beginning (and will continue to be done throughout this section) is that SQL Server provides no 'Are you sure?' option.  Once you UPDATE the data there is no going back outside of expensive database restores.  (This CAN be handled with additional, more complex syntax and methodologies, but these are not covered within the basics as they have wide ranging implications when done incorrectly and without more knowledge of the internals of SQL Server such as Locking and Blocking, Transactions, and Isolation Levels)

It is also worth noting that our demo scripts will start to become quite lengthy in this section as we need to perform the following for each demo:
* Drop a temporary table if it exists
* Create a temporary table
* Populate the temporary table
* Perform an Update

However, this is vital in order to be able to understand how the UPDATE syntax works.

#### Basic Update

We shall start with a basic update.  Simply changing a single value to be something else.

To do this we shall create a basic temporary table with key Employee information, populate it with just 1 record, and then we shall perform our update.

Firstly let's look at our demo setup:

In [ ]:
drop table if exists #employeeData;

create table #employeeData
(
	employeeID int,
	firstName varchar(25),
	lastName varchar(35),
	dateOfBirth date,
	salary decimal(10, 2)
);

insert into #employeeData
(
	employeeID, firstName, lastName, dateOfBirth, salary
)
select employeeID, firstName, lastName, dob, salary
from JupyterDatabase.hr.employee
where employeeID = 1;

select *
from #employeeData;

Let's now imagine that we want to give this employee a 10% payrise. How do we do that?

In the most basic form of the UPDATE statement this is a very simple task as as we refer back to SQL being a declarative language and we 'UPDATE our table in order to SET one value to equal another value WHERE our criteria is met':

<code>UPDATE <i>tableName</i><br/>
SET <i>currentValueOrColumn</i> = <i>newValueOrColumn</i><br/>
WHERE <i>column</i> = <i>value</i></code>

Let's alter the date of birth for our employee, changing it to the static value '1979-01-01':

In [ ]:
drop table if exists #employeeData;

create table #employeeData
(
	employeeID int,
	firstName varchar(25),
	lastName varchar(35),
	dateOfBirth date,
	salary decimal(10, 2)
);

insert into #employeeData
(
	employeeID, firstName, lastName, dateOfBirth, salary
)
select employeeID, firstName, lastName, dob, salary
from JupyterDatabase.hr.employee
where employeeID = 1;

-- Check on the data we've loaded
select *
from #employeeData;

-------------------------------------------------------------
-- Perform the UPDATE
-------------------------------------------------------------
update #employeeData
set dateOfBirth = '1979-01-01'
where employeeID = 1;
-------------------------------------------------------------

-- Check on the data we've updated
select *
from #employeeData;

In the above example we only have 1 record in our #employeeData table and therefore the WHERE clause is not strictly necessary but as it is best practice to have a WHERE clause we shall get into the habit from the start.

In the above example we updated a column with a static value.  This is unlikely to always be the case.  For example, imagine if we have 5 employees and wish to give employeeIDs 1, 2, and 3 a 10% payrise.  We can no longer achieve this using a static value in our UPDATE statement.

Luckily we can also use a column as a source for our update:

In [ ]:
drop table if exists #employeeData;

create table #employeeData
(
	employeeID int,
	firstName varchar(25),
	lastName varchar(35),
	dateOfBirth date,
	salary decimal(10, 2)
);

insert into #employeeData
(
	employeeID, firstName, lastName, dateOfBirth, salary
)
select employeeID, firstName, lastName, dob, salary
from JupyterDatabase.hr.employee
where employeeID <= 5;

-- Check on the data we've loaded
select *
from #employeeData;

-------------------------------------------------------------
-- Perform the UPDATE
-------------------------------------------------------------
update #employeeData
set salary = salary * 1.1
where employeeID <= 3;
-------------------------------------------------------------

-- Check on the data we've updated
select *
from #employeeData;

Now we can understand the Basic Update statement we shall tweak it slightly in order to make it more adaptable and useful in the future once we move into more complex areas.

#### UPDATE with an Alias

Remembering back to our basic SELECT statements we learnt to write code using basic syntax and then gradually moved on to adding table aliases into our queries as this opened up new avenues such as the ability to JOIN.

We are going to follow the same progression with the UPDATE statement in that we will now introduce aliasing in order to allow for more complex updates to be discussed later, such as Updating via a Join.

Using an alias does slightly alter the syntax that we use but actually makes the code look more like the SQL we have learnt to date:

<code>UPDATE <i>tableAlias</i><br/>
SET <i>currentValueOrColumn</i> = <i>newValueOrColumn</i><br/>
FROM <i>myTable</i> AS <i>tableAlias</i><br/>
WHERE <i>column</i> = <i>value</i></code>

The above template shows how we are using FROM to specify our table and create our table alias which we can then pass to the UPDATE keyword in order to tell it which table it is updating.

Let's reproduce our first example above (altering the date of birth of our 1 employee) but using the newer, more complete, syntax:

In [ ]:
drop table if exists #employeeData;

create table #employeeData
(
	employeeID int,
	firstName varchar(25),
	lastName varchar(35),
	dateOfBirth date,
	salary decimal(10, 2)
);

insert into #employeeData
(
	employeeID, firstName, lastName, dateOfBirth, salary
)
select employeeID, firstName, lastName, dob, salary
from JupyterDatabase.hr.employee
where employeeID = 1;

-- Check on the data we've loaded
select *
from #employeeData;

-------------------------------------------------------------
-- Perform the UPDATE
-------------------------------------------------------------
update e
set dateOfBirth = '1979-01-01'
from #employeeData as e
where employeeID = 1;
-------------------------------------------------------------

-- Check on the data we've updated
select *
from #employeeData;

We can also use the new syntax in order to perform our salary increases, however, we are going to make this variant a little different to highlight that we are not bound by static values or basic maths in our updates, but can use all tools available to us, for example, the CASE statement.

We are going to use the CASE statement to specify the following:
* For all top 10 employees
* Increase their salary by:
 * WHEN salary >= 1000000 then 5%
 * WHEN salary >= 500000 then 10%
 * WHEN salary >= 250000 then 15%
 * ELSE 20%

From the above we know that our CASE statement will be as follows:

<code>CASE WHEN salary >= 1000000 then 1.05<br/>
&nbsp&nbsp&nbsp&nbsp&nbspWHEN salary >= 500000 then 1.1<br/>
&nbsp&nbsp&nbsp&nbsp&nbspWHEN salary >= 250000 then 1.15<br/>
&nbsp&nbsp&nbsp&nbsp&nbspELSE 1.2<br/>
END</code>

So let's combine that with our UPDATE using the fuller alias syntax:

In [ ]:
drop table if exists #employeeData;

create table #employeeData
(
	employeeID int,
	firstName varchar(25),
	lastName varchar(35),
	dateOfBirth date,
	salary decimal(10, 2)
);

insert into #employeeData
(
	employeeID, firstName, lastName, dateOfBirth, salary
)
select employeeID, firstName, lastName, dob, salary
from JupyterDatabase.hr.employee
where employeeID <= 10;

-- Check on the data we've loaded
select *
from #employeeData;

-------------------------------------------------------------
-- Perform the UPDATE
-------------------------------------------------------------
update e
set salary = salary *
            (
                case when salary >= 1000000 then 1.05
                     when salary >= 500000 then 1.1
                     when salary >= 250000 then 1.15
                else 1.2 end
            )
from #employeeData as e
where employeeID <= 10;
-------------------------------------------------------------

-- Check on the data we've updated
select *
from #employeeData;

Using this alias syntax it is clear how we can start to utilise JOINs in order to perform more wide ranging UPDATEs on our data.

We will discuss and perform these shortly.